# Modifications de résultats des sondages

## Import préliminaires des librairies

In [13]:
import re
import numpy as np
import pandas as pd
import locale
from locale import atof

# Read files
import PyPDF2
from pdfminer.high_level import extract_text
import camelot as km

## Extraction en utilisant camelot

In [41]:
fp="rapport_personnalite/2021-02-11.pdf"
tables = km.read_pdf(fp,kind="textedge", process_background=True,pages='{}-{}'.format(12,13))

In [42]:
tables[0].df

,0,1,2,3,4,5,6,7,8
0,Nicolas \nEdouard \nRoselyne \nBruno Le \nMart...,,,,,,,,
1,,ENSEMBLE ........................................,65,61,52,52,51,50,48
2,,,,,,,,,
3,,,,,,,,,
4,SEXE DE L'INTERVIEWE(E),,,,,,,,
...,...,...,...,...,...,...,...,...,...
90,,,,,,,,,
91,,,,,,,,,
92,,,,,,,,,
93,,,,,,,,,


## Extraction utilisant une lecture de PDF

In [2]:
file_path="rapport_personnalite/2021-02-11.pdf"

In [3]:

text = extract_text(file_path,page_numbers=[11])

print(text)


 

Le classement général 

TOTAL BONNE opinion 

Nicolas 
Hulot 

Edouard 
Philippe 

Roselyne 
Bachelot 

Bruno Le 
Maire 

Martine 
Aubry 

Nicolas 
Sarkozy 

Emmanuel 
Macron 

Jean 
Castex 

67 
ENSEMBLE ........................................................................................................  

61 

65 

52 

52 

51 

50 

48 

  

  

  

  

  

  

  

  

  

  

  

  

  

  

  

  

  

  

  

  

58 
47 

63 
67 

60 
62 

43 
42 
43 
56 
49 
50 
70 

57 
63 
53 
67 
64 
64 
76 

57 
57 
50 
46 
55 
60 
32 

52 
80 
59 
65 
57 
69 
58 

49 
47 
49 
65 
62 
61 
73 

51 
65 
59 
66 
58 
66 
51 

SEXE DE L'INTERVIEWE(E) 
Homme .................................................................................................................  
50 
69 
Femme ..................................................................................................................  
55 
65 
AGE DE L'INTERVIEWE(E) 
TOTAL Moins de 35 ans ....................................

In [62]:
with open(file_path,"rb") as pdf_file:
        pdf_reader= PyPDF2.PdfFileReader(pdf_file)
        page_content=pdf_reader.getPage(11).extractText()
# print(get_base(file_path))
# pretraitement(page_content)
page_content


" \nIfop\n-\nFiducial\n \npour \nParis Match et Sud Radio\n \nLe tableau de bord des personnalités\n \n\n \nMars 2021\n \n10\n \nLe classement \ngénéral\n \nTOTAL BONNE opinion\n \nNicolas \nHulot\n \nEdouard \nPhilippe\n \nRoselyne \nBachelot\n \nBruno Le \nMaire\n \nMartine \nAubry\n \nEmmanuel \nMacron\n \nFrançois \nHollande\n \nJean Castex\n \nENSEMBLE\n \n................................\n................................\n................................\n................................\n................................\n................................\n................................\n................................\n................................\n................................\n................................\n................................\n................................\n................................\n................................\n................................\n................................\n...........\n \n \n66\n \n62\n \n58\n \n51\n \n47\n \n46\n

In [76]:
def pretraitement(text):
    text=text.replace('opinion\n \n»','opinion\n \n')
    text=text.replace('BONNE \n','BONNE ')
    return text

In [78]:
def get_names(file_path,page):
    with open(file_path,"rb") as pdf_file:
        pdf_reader= PyPDF2.PdfFileReader(pdf_file)
        page_content=pdf_reader.getPage(page).extractText()
    page_content=pretraitement(page_content)
    text=page_content[page_content.lower().find('bonne opinion'):]
    print(f"POSITION {page_content.lower().find('bonne opinion')}")
    text=text.replace('\n \n','\n\n')
    text=text.split('\n\n')[1:]
    noms=[]
    for i in range(text.index('ENSEMBLE')):
        nom=text[i]
        nom=nom.replace('\n','')
        print(nom)
        fullname=nom.split(' ')
        fullname[1]=''.join([x.upper() for x in fullname[1:]])
        nom=' '.join(fullname[:2])
        noms.append(nom)
    return noms
# get_names(file_path,11)

In [7]:
def get_pos_opinions(file_path,page,nb_pers):
    with open(file_path,"rb") as pdf_file:
        pdf_reader= PyPDF2.PdfFileReader(pdf_file)
        page_content=pdf_reader.getPage(page).extractText()
    page_content=pretraitement(page_content)
    text=page_content[page_content.lower().find('bonne opinion'):]
    opinions=[int(x) for x in re.findall('\d+',text)[:nb_pers]]
    return opinions
get_pos_opinions(file_path,11,8)
    

[67, 65, 61, 52, 52, 51, 50, 48]

In [8]:
def get_base(file_path,page=3):
    text = extract_text(file_path,page_numbers=[page])
    txt_idx="auprès d’un échantillon de "
    idx=text.find(txt_idx)+len(txt_idx)
    return float(re.findall('[0-9]+',text[idx:idx+10].replace(' ',''))[0])

In [9]:
def extract_ifop_opinions(file_path,p_start,p_end):
    base=get_base(file_path)
    date=file_path.split("/")[-1].rstrip('.pdf')
    columns=['Nom','Base','Date','Opinion positive']
    df=pd.DataFrame(columns=columns)
    for p_nb in range(p_start,p_end+1):
        noms=get_names(file_path,p_nb)
        print(noms)
        opinions=get_pos_opinions(file_path,p_nb,len(noms))
        c_df=pd.DataFrame(columns=columns)
        c_df['Nom']=noms
        c_df['Opinion positive']=opinions
        c_df['Date']=date
        c_df['Base']=base
        df=pd.concat([df,c_df],ignore_index=True)
    return df

In [79]:
filename="2021-03-04"
file_path='rapport_personnalite/'+filename+'.pdf'
p_start,p_end=11,16
# Enlever 1 par rapport au pdf aux pages
df=extract_ifop_opinions(file_path,p_start,p_end)
df.to_pickle('data_perso/'+filename+'.p')
df

POSITION 143
Nicolas Hulot
Edouard Philippe
Roselyne Bachelot
Bruno Le Maire
Martine Aubry
Emmanuel Macron
François Hollande
Jean Castex
['Nicolas HULOT', 'Edouard PHILIPPE', 'Roselyne BACHELOT', 'Bruno LEMAIRE', 'Martine AUBRY', 'Emmanuel MACRON', 'François HOLLANDE', 'Jean CASTEX']
POSITION 142
Olivier Véran
Xavier Bertrand
Jean-Yves Le Drian
Rachida Dati
François Baroin
Bernard Cazeneuve
Nicolas Sarkozy
Valérie Pécresse
Jean-Michel Blanquer
['Olivier VÉRAN', 'Xavier BERTRAND', 'Jean-Yves LEDRIAN', 'Rachida DATI', 'François BAROIN', 'Bernard CAZENEUVE', 'Nicolas SARKOZY', 'Valérie PÉCRESSE', 'Jean-Michel BLANQUER']
POSITION 142
Anne Hidalgo
François Bayrou
Eric Dupond-Moretti
Arnaud Montebourg
Ségolène Royal
Jean-Luc Mélenchon
Laurent Wauquiez
Gérard Larcher
Gabriel Attal
['Anne HIDALGO', 'François BAYROU', 'Eric DUPOND-MORETTI', 'Arnaud MONTEBOURG', 'Ségolène ROYAL', 'Jean-Luc MÉLENCHON', 'Laurent WAUQUIEZ', 'Gérard LARCHER', 'Gabriel ATTAL']
POSITION 142
Christian Estrosi
Gérald Da

,Nom,Base,Date,Opinion positive
0,Nicolas HULOT,1006.0,2021-03-04,66
1,Edouard PHILIPPE,1006.0,2021-03-04,62
2,Roselyne BACHELOT,1006.0,2021-03-04,58
3,Bruno LEMAIRE,1006.0,2021-03-04,51
4,Martine AUBRY,1006.0,2021-03-04,47
5,Emmanuel MACRON,1006.0,2021-03-04,46
6,François HOLLANDE,1006.0,2021-03-04,45
7,Jean CASTEX,1006.0,2021-03-04,45
8,Olivier VÉRAN,1006.0,2021-03-04,43
9,Xavier BERTRAND,1006.0,2021-03-04,43


In [29]:
# import datetime
# df=pd.read_pickle('data/2021-01-21.p')
# df.loc[0,df.columns[4:13]]=[7,34,41,34,25,59,0,-4,+4]
# df.loc[1,df.columns[4:13]]=[4,34,38,34,28,62,0,-3,+3]
# df.Base=1004
# df.Date=datetime.date(2021,2,5)

In [32]:
# df.to_pickle('data/2021-02-05.p')

In [29]:
# df.to_pickle('data/'+filename+'.p')

In [ ]:
# df3=pd.concat([df,df2],ignore_index=True)
# df3.to_pickle('data/'+filename+'.p')

In [11]:
# Reading IPSOS report

# with open("rapport/2020-10-14.pdf","rb") as pdf_file:
#     pdf_report= PyPDF2.PdfFileReader(pdf_file)
#     isEncrypted = pdf_report.getIsEncrypted()
#     nbPages= pdf_report.numPages

#     page1=pdf_report.getPage(15)
#     text=page1.extractText().replace(',','.')
#     text = extract_text(pdf_file,page_numbers=[14])

# print(text)

print(df.isnull().values.any())
df.head(10)
df.describe()
df[df.Base==0]

False


,Nom,Base,Date,Opinion positive


In [12]:
df

,Nom,Base,Date,Opinion positive
0,Nicolas HULOT,1005.0,2021-02-11,67
1,Edouard PHILIPPE,1005.0,2021-02-11,65
2,Roselyne BACHELOT,1005.0,2021-02-11,61
3,Bruno LEMAIRE,1005.0,2021-02-11,52
4,Martine AUBRY,1005.0,2021-02-11,52
5,Nicolas SARKOZY,1005.0,2021-02-11,51
6,Emmanuel MACRON,1005.0,2021-02-11,50
7,Jean CASTEX,1005.0,2021-02-11,48
8,Jean-Yves LEDRIAN,1005.0,2021-02-11,48
9,Valérie PÉCRESSE,1005.0,2021-02-11,46
